In [78]:

import pandas as pd
import numpy as np
from datetime import datetime 
import requests
import math
import re

from bs4 import BeautifulSoup

In [79]:
# url do site
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

# dicionario que diz para i end point que quem faz a requisicao e um chorme (Browser)
headers ={''}
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

# Requisita essa url do servidor do site H&M
page = requests.get( url, headers=headers )

# Product lista das primeiras coletas id, cat, name, price
soup = BeautifulSoup( page.text, 'html.parser' )

# Toda minha vitrine
products = soup.find('ul', class_= 'products-listing small')



In [80]:
# Todos meu produtos da vitrine e sua cat, onde filtro todas que preciso
product_list = products.find_all('article', class_ = 'hm-product-item')

# Product_id
product_list[0].get('data-articlecode')
product_id = [i.get('data-articlecode') for i in product_list]

# Product_category
product_list[0].get('data-category')
product_category = [i.get('data-category') for i in product_list]

# product_name
product_list = products.find_all('a', class_= 'item-link' )
product_list[0].get('title')
product_name = [i.get('title') for i in product_list]

# product_price
product_list = products.find_all('span', class_='price regular')
product_price = [i.get_text() for i in product_list]

In [81]:
# Criando o data frame
data = pd.DataFrame([ product_id, product_name, product_category, product_price ]).T
#colunas do DataFrame
data.columns = ['product_id', 'product_name', 'product_category', 'product_price']

# scrapy datetime
data['scrapy_datetime'] = datetime.now().strftime( '%Y-%m-%d %H:%M:%S:')
# plotar
data.head()

,product_id,product_name,product_category,product_price,scrapy_datetime
0,1024256001,Slim Jeans,men_jeans_slim,$ 24.99,2023-10-04 20:19:16:
1,1130309005,Baggy Jeans,men_jeans_loose,$ 39.99,2023-10-04 20:19:16:
2,0979945001,Loose Jeans,men_jeans_loose,$ 39.99,2023-10-04 20:19:16:
3,0985159001,Skinny Jeans,men_jeans_skinny,$ 29.99,2023-10-04 20:19:16:
4,1008549001,Straight Regular Jeans,men_jeans_regular,$ 29.99,2023-10-04 20:19:16:


# Coleta para um Unico Produto

In [82]:

url = 'https://www2.hm.com/en_us/productpage.0985159001.html'

# dicionario que diz para i end point que quem faz a requisicao e um chorme (Browser)
headers ={''}
headers = headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
# Requisita essa url do servidor do site H&M
page = requests.get( url, headers=headers )

# ================  One Product =================================

# para acesar a pagina html do page , objeto da classe request
page.text
# Parametro de entrada para o beautifulsoup
soup = BeautifulSoup( page.text, 'html.parser' )

# ========================== Color Name ===============================
# Coletar a cor color_name
product_list = soup.find_all('a', class_='filter-option miniature')
product_color = [i.get('data-color') for i in product_list]

# product_id
product_id = [i.get('data-articlecode') for i in product_list]

# Ccriando o data frame
df_color = pd.DataFrame([product_id, product_color]).T
df_color.columns = ['product_id', 'color_name']

# # generate style id+ color id
df_color['style_id'] = df_color['product_id'].apply( lambda x: x[:-3] )
df_color['color_id'] = df_color['product_id'].apply( lambda x: x[-3:] )

# =============================== Caategory =======================================

# Encontre todos os elementos que representam os produtos
product_elements = soup.find_all('div', class_='content pdp-text pdp-content')

# Inicialize uma lista para armazenar as informações de todos os produtos
all_products_info = []

for product_element in product_elements:
    product_info = {}
    
    # Extrair informações específicas para cada produto
    fit_element = product_element.find('dt', string='Fit')
    if fit_element:
        fit = fit_element.find_next('dd').text.strip()
        product_info['Fit'] = fit
        
    size_element = product_element.find('dt', string='Size')
    if size_element:
        size = size_element.find_next('dd').text.strip()
        product_info['Size'] = size    
    
    composition_element = product_element.find('h3', string='Composition')
    if composition_element:
        composition = composition_element.find_next('ul').text.strip()
        product_info['Composition'] = composition.replace('\n', '')
        
    article_number_element = product_element.find('div', string=re.compile(r'Article number\d+'))
    if article_number_element:
        article_number_text = article_number_element.text.strip()
        article_number = re.search(r'\d+', article_number_text).group()
        product_info['Article number'] = article_number    
        
    # Adicionar as informações do produto à lista
    all_products_info.append(product_info);

# criar data frame 
df_composition = pd.DataFrame(all_products_info)

df_composition['style_id'] = df_composition['Article number'].apply( lambda x: x[:-3])
df_composition['color_id'] = df_composition['Article number'].apply( lambda x: x[-3:])

# ========================== df_color + df_composition ================================== 
data_sku = pd.merge( df_color, df_composition[['style_id', 'Fit', 'Composition']], how='left', on='style_id')

# Requisicao para toda a vitrine

In [88]:
# dicionario que diz para i end point que quem faz a requisicao e um chorme (Browser)
headers ={''}
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

df_final = pd.DataFrame()

# cria a lista vazia para colocar as col que nao tenho aqui coletada caso meu novo prod tenha alguma caracteristca que nao peguei no promeiro
aux = []

cols = ['Article number','Composition','Fit','Pocket lining','color_id','style_id','Size']
df_patter = pd.DataFrame(columns=cols)

for i in range( len( data ) ):
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'
    print(url)
    # Requisita essa url do servidor do site H&M
    page = requests.get( url, headers=headers )
    
    # Parametro de entrada para o beautifulsoup
    soup = BeautifulSoup( page.text, 'html.parser' )

    # ========================== Color Name ===============================
    # Coletar a cor color_name
    product_list = soup.find_all('a', class_='filter-option miniature')
    product_color = [i.get('data-color') for i in product_list]

    # product_id
    product_id = [i.get('data-articlecode') for i in product_list]

    #    Ccriando o data frame
    df_color = pd.DataFrame([product_id, product_color]).T
    df_color.columns = ['product_id', 'color_name']

    # # generate style id+ color id
    df_color['style_id'] = df_color['product_id'].apply( lambda x: x[:-3] )
    df_color['color_id'] = df_color['product_id'].apply( lambda x: x[-3:] )

    # =============================== Caategory =======================================

    # Encontrei todos os elementos que representam os produtos
    product_elements = soup.find_all('div', class_='content pdp-text pdp-content')

    # Inicializei uma lista para armazenar as informações de todos os produtos
    all_products_info = []

    for product_element in product_elements:
        product_info = {}
    
        # Extrair informações específicas para cada produto
        fit_element = product_element.find('dt', string='Fit')
        if fit_element:
            fit = fit_element.find_next('dd').text.strip()
            product_info['Fit'] = fit
        
        size_element = product_element.find('dt', string='Size')
        if size_element:
            size = size_element.find_next('dd').text.strip()
            product_info['Size'] = size    
    
        composition_element = product_element.find('h3', string='Composition')
        if composition_element:
            composition = composition_element.find_next('ul').text.strip()
            product_info['Composition'] = composition.replace('\n', '')
        
        article_number_element = product_element.find('div', string=re.compile(r'Article number\d+'))
        if article_number_element:
            article_number_text = article_number_element.text.strip()
            article_number = re.search(r'\d+', article_number_text).group()
            product_info['Article number'] = article_number    
        
        # Adicionar as informações do produto à lista
        all_products_info.append(product_info)
        
    # criar data frame 
    df_composition = pd.DataFrame(all_products_info)
    
        
    # Garante the same number o columns    
    df_composition = pd.concat([ df_patter, df_composition], axis=0)
    
    
    df_composition['style_id'] = df_composition['Article number'].apply( lambda x: x[:-3])
    df_composition['color_id'] = df_composition['Article number'].apply( lambda x: x[-3:])
    
    # guardo aqui minhas col de composition e concateno a lista somando
    aux = aux + df_composition.columns.tolist()

    # ========================== df_color + df_composition ================================== 
    if 'Size' in df_composition.columns:
       data_sku = pd.merge(df_color, df_composition[['style_id', 'Fit', 'Composition', 'Size']], how='left', on='style_id')
    else:
       data_sku = pd.merge(df_color, df_composition[['style_id', 'Fit', 'Composition']], how='left', on='style_id')

    
    #data_sku = pd.merge( df_color, df_composition[['style_id', 'Fit', 'Composition','Pocket lining', 'Size']], how='left', on='style_id')
    
    df_final = pd.concat([df_final, data_sku], axis=0)
    
# Join showroom data + df_final
    
data['style_id'] = data['product_id'].apply( lambda x: x[:-3])
data['color_id'] = data['product_id'].apply( lambda x: x[-3:])

data_raw = pd.merge(data, df_final[['style_id','color_name', 'Fit', 'Composition', 'Size']], how='left', on='style_id')    

https://www2.hm.com/en_us/productpage.1024256001.html


https://www2.hm.com/en_us/productpage.1130309005.html
https://www2.hm.com/en_us/productpage.0979945001.html
https://www2.hm.com/en_us/productpage.0985159001.html
https://www2.hm.com/en_us/productpage.1008549001.html
https://www2.hm.com/en_us/productpage.1130309004.html
https://www2.hm.com/en_us/productpage.1182102002.html
https://www2.hm.com/en_us/productpage.1024256011.html
https://www2.hm.com/en_us/productpage.1159764002.html
https://www2.hm.com/en_us/productpage.0979945003.html
https://www2.hm.com/en_us/productpage.1008549005.html
https://www2.hm.com/en_us/productpage.1024256006.html
https://www2.hm.com/en_us/productpage.1202825001.html
https://www2.hm.com/en_us/productpage.1096385001.html
https://www2.hm.com/en_us/productpage.1130309007.html
https://www2.hm.com/en_us/productpage.1008549006.html
https://www2.hm.com/en_us/productpage.1182102001.html
https://www2.hm.com/en_us/productpage.1166422001.html
https://www2.hm.com/en_us/productpage.0985159004.html
https://www2.hm.com/en_us/pr

In [89]:
data_raw

,product_id,product_name,product_category,product_price,scrapy_datetime,style_id,color_id,color_name,Fit,Composition,Size
0,1024256001,Slim Jeans,men_jeans_slim,$ 24.99,2023-10-04 20:19:16:,1024256,001,Light denim blue,Slim fit,"ShellCotton 99%, Spandex 1%Pocket liningPolyes...",NaN
1,1024256001,Slim Jeans,men_jeans_slim,$ 24.99,2023-10-04 20:19:16:,1024256,001,Light denim blue,Slim fit,"ShellCotton 99%, Spandex 1%Pocket liningPolyes...",NaN
2,1024256001,Slim Jeans,men_jeans_slim,$ 24.99,2023-10-04 20:19:16:,1024256,001,Denim blue,Slim fit,"ShellCotton 99%, Spandex 1%Pocket liningPolyes...",NaN
3,1024256001,Slim Jeans,men_jeans_slim,$ 24.99,2023-10-04 20:19:16:,1024256,001,Dark blue,Slim fit,"ShellCotton 99%, Spandex 1%Pocket liningPolyes...",NaN
4,1024256001,Slim Jeans,men_jeans_slim,$ 24.99,2023-10-04 20:19:16:,1024256,001,Dark denim blue,Slim fit,"ShellCotton 99%, Spandex 1%Pocket liningPolyes...",NaN
...,...,...,...,...,...,...,...,...,...,...,...
906,1166422006,Tapered Regular Crop Jeans,men_jeans_tapered,$ 34.99,2023-10-04 20:19:16:,1166422,006,Black,Regular fit,"ShellCotton 99%, Spandex 1%Pocket liningPolyes...",Inner leg: Length: 67.5 cm (Size 33)
907,1166422006,Tapered Regular Crop Jeans,men_jeans_tapered,$ 34.99,2023-10-04 20:19:16:,1166422,006,Light denim blue,Regular fit,"ShellCotton 98%, Spandex 2%Pocket liningPolyes...","The model is 189cm/6'2"" and wears a size 32"
908,1166422006,Tapered Regular Crop Jeans,men_jeans_tapered,$ 34.99,2023-10-04 20:19:16:,1166422,006,White,Regular fit,"ShellCotton 98%, Spandex 2%Pocket liningPolyes...","The model is 189cm/6'2"" and wears a size 32"
909,1166422006,Tapered Regular Crop Jeans,men_jeans_tapered,$ 34.99,2023-10-04 20:19:16:,1166422,006,Denim gray,Regular fit,"ShellCotton 98%, Spandex 2%Pocket liningPolyes...","The model is 189cm/6'2"" and wears a size 32"


In [90]:
data_raw['Composition'].unique()

array(['ShellCotton 99%, Spandex 1%Pocket liningPolyester 65%, Cotton 35%',
       'ShellCotton 99%, Spandex 1%Pocket liningCotton 100%',
       'ShellCotton 100%Pocket liningPolyester 65%, Cotton 35%',
       'ShellCotton 98%, Spandex 2%Pocket liningPolyester 65%, Cotton 35%',
       'ShellCotton 99%, Spandex 1%PocketCotton 100%', 'Cotton 100%', nan,
       'ShellCotton 100%Pocket liningCotton 100%',
       'ShellCotton 99%, Spandex 1%Pocket liningPolyester 80%, Cotton 20%',
       'Cotton 98%, Spandex 2%LiningPolyester 100%',
       'Cotton 98%, Spandex 2%', 'ShellCotton 99%, Spandex 1%',
       'ShellCotton 90%, Elastomultiester 8%, Spandex 2%Pocket liningPolyester 80%, Cotton 20%'],
      dtype=object)

In [91]:
# criando um csv
colection = 'data_webs.csv'
data_raw.to_csv(colection, index=False)

In [92]:

print(data.shape)
print(df_final.shape)
print(data_raw.shape)

(36, 7)
(324, 7)
(911, 11)
